In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from collections import defaultdict
import mysql.connector

# Function to convert American odds to implied probability
def american_odds_to_probability(odds_str):
    odds = int(odds_str.replace('+', ''))
    if odds > 0:
        implied_prob = 100 / (odds + 100)
    else:
        implied_prob = abs(odds) / (abs(odds) + 100)
    return implied_prob

# Set up the WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

try:
    # Open the ESPN NFL Futures page for Conference odds
    driver.get('https://www.espn.com/nfl/futures/_/group/conference')
    driver.implicitly_wait(10)

    # Define dictionary to hold odds for each team and category (Super Bowl and Conference)
    team_odds = {
        'Kansas City Chiefs': {'conference_odds': None, 'super_bowl_odds': None},
        'Baltimore Ravens': {'conference_odds': None, 'super_bowl_odds': None}
    }

    # Locate the conference odds table by finding the AFC Champion section
    rows = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='betFutures'] table[data-testid='prism-Table'] tbody tr")

    # Scrape conference odds
    for row in rows:
        team_name = row.find_element(By.CSS_SELECTOR, "a[data-testid='prism-linkbase'] span").text.strip()
        odds = row.find_elements(By.CSS_SELECTOR, "a[data-testid='prism-linkbase']")[-1].text.strip()
        
        if team_name in team_odds:
            team_odds[team_name]['conference_odds'] = odds
            print(f"{team_name} Conference Odds: {odds}")

    # Switch to Super Bowl odds
    driver.get('https://www.espn.com/nfl/futures/_/group/superbowl')
    driver.implicitly_wait(10)

    # Scrape Super Bowl odds
    rows = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='betFutures'] table[data-testid='prism-Table'] tbody tr")
    for row in rows:
        team_name = row.find_element(By.CSS_SELECTOR, "a[data-testid='prism-linkbase'] span").text.strip()
        odds = row.find_elements(By.CSS_SELECTOR, "a[data-testid='prism-linkbase']")[-1].text.strip()
        
        if team_name in team_odds:
            team_odds[team_name]['super_bowl_odds'] = odds
            print(f"{team_name} Super Bowl Odds: {odds}")

finally:
    driver.quit()

# Convert odds to implied probabilities
for team, odds in team_odds.items():
    if odds['conference_odds']:
        odds['conference_prob'] = american_odds_to_probability(odds['conference_odds'])
    if odds['super_bowl_odds']:
        odds['super_bowl_prob'] = american_odds_to_probability(odds['super_bowl_odds'])

# Output probabilities for verification
for team, odds in team_odds.items():
    print(f"{team} - Conference Probability: {odds.get('conference_prob', 'N/A'):.2%}, Super Bowl Probability: {odds.get('super_bowl_prob', 'N/A'):.2%}")

# Connect to MySQL database and fetch potential payouts
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get Potential Payouts for Conf, Champ
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Fetch results and organize them in a dictionary
payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0})

# Define relevant teams
relevant_teams = ['Kansas City Chiefs', 'Baltimore Ravens']

# Fetch and organize payouts by relevant teams
for participant_name, event_type, total_payout in cursor.fetchall():
    if participant_name in relevant_teams:
        if event_type == 'Conference Winner':
            payouts[participant_name]['payout_conference'] = float(total_payout)
        elif event_type == 'Championship':
            payouts[participant_name]['payout_championship'] = float(total_payout)

cursor.close()
conn.close()

# Define the EV calculation functions
def calculate_ev(probability, payout):
    """Calculate expected value based on probability and payout."""
    return probability * payout

def calculate_conditional_ev(current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    """Calculate conditional EV if the team wins the current round."""
    conditional_conference_prob = current_conference_prob / current_round_prob
    conditional_champ_prob = current_champ_prob / current_round_prob
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)
    return conference_ev + champ_ev

# Inputs for the two teams
team1 = {
    "current_round_prob": 0.534,  # Manually input this probability
    "current_conference_prob": team_odds['Kansas City Chiefs']['conference_prob'],  
    "current_champ_prob": team_odds['Kansas City Chiefs']['super_bowl_prob'],
    "conference_payout": payouts['Kansas City Chiefs']['payout_conference'], 
    "champ_payout": payouts['Kansas City Chiefs']['payout_championship'] 
}

team2 = {
    "current_round_prob": 1 - team1["current_round_prob"],  # Complement probability
    "current_conference_prob": team_odds['Baltimore Ravens']['conference_prob'],  
    "current_champ_prob": team_odds['Baltimore Ravens']['super_bowl_prob'],
    "conference_payout": payouts['Baltimore Ravens']['payout_conference'], 
    "champ_payout": payouts['Baltimore Ravens']['payout_championship'] 
}

# Calculate conditional EVs if each team wins the current round
team1_ev_win = calculate_conditional_ev(
    team1["current_conference_prob"], team1["conference_payout"],
    team1["current_champ_prob"], team1["champ_payout"],
    team1["current_round_prob"]
)

team2_ev_win = calculate_conditional_ev(
    team2["current_conference_prob"], team2["conference_payout"],
    team2["current_champ_prob"], team2["champ_payout"],
    team2["current_round_prob"]
)

# Calculate the Series Delta as the difference between conditional EVs
series_delta = abs(team1_ev_win - team2_ev_win)

# Output results
print("\nConditional Expected Values if Team Wins Current Round:")
print(f"Kansas City Chiefs EV if win round: ${team1_ev_win:.2f}")
print(f"Baltimore Ravens EV if win round: ${team2_ev_win:.2f}")
print("\nSeries Delta:")
print(f"${series_delta:.2f}")


Kansas City Chiefs Conference Odds: +160
Baltimore Ravens Conference Odds: +275
Kansas City Chiefs Super Bowl Odds: +375
Baltimore Ravens Super Bowl Odds: +550
Kansas City Chiefs - Conference Probability: 38.46%, Super Bowl Probability: 21.05%
Baltimore Ravens - Conference Probability: 26.67%, Super Bowl Probability: 15.38%

Conditional Expected Values if Team Wins Current Round:
Kansas City Chiefs EV if win round: $42487.68
Baltimore Ravens EV if win round: $14196.10

Series Delta:
$28291.58


In [4]:
## Playoffs Individual Game EV Calculator (Isolated to the Matchup)


def calculate_team_ev(new_series_odds, series_ev):
    """Calculate the team EV based on new series odds and series EV."""
    return new_series_odds * series_ev

def calculate_game3_ev_delta(team1_game3_win_prob, team2_game3_win_prob, team1_new_series_odds, team2_new_series_odds, team1_ev_win, team2_ev_win):
    """Calculate the EV Delta for Game 3."""
    
    # Calculate EV if Team 1 wins Game 3
    team1_ev_if_win = calculate_team_ev(team1_new_series_odds, team1_ev_win)
    team2_ev_if_lose = calculate_team_ev(1 - team1_new_series_odds, team2_ev_win)
    ev_if_team1_wins_game3 = team1_ev_if_win + team2_ev_if_lose
    
    # Calculate EV if Team 2 wins Game 3
    team1_ev_if_lose = calculate_team_ev(1 - team2_new_series_odds, team1_ev_win)
    team2_ev_if_win = calculate_team_ev(team2_new_series_odds, team2_ev_win)
    ev_if_team2_wins_game3 = team1_ev_if_lose + team2_ev_if_win
    
    # Calculate the EV Delta for Game 3
    ev_delta = abs(ev_if_team1_wins_game3 - ev_if_team2_wins_game3)
    
    return ev_if_team1_wins_game3, ev_if_team2_wins_game3, ev_delta

# Inputs specific to Game 3
team1_game3_win_prob = 0.53  # Team 1's probability of winning Game 3
team2_game3_win_prob = (1 - team1_game3_win_prob)  # Team 2's probability of winning Game 3
team1_new_series_odds = 1  # Series odds if Team 1 wins Game 3
team2_new_series_odds = 0.99148936170  # Series odds if Team 2 wins Game 3

# Use team1_ev_win and team2_ev_win directly from the previous script
# Calculate EV if Team 1 or Team 2 wins Game 3 and the EV Delta
ev_if_team1_wins_game3, ev_if_team2_wins_game3, game3_ev_delta = calculate_game3_ev_delta(
    team1_game3_win_prob,
    team2_game3_win_prob,
    team1_new_series_odds,
    team2_new_series_odds,
    team1_ev_win,  # From previous script
    team2_ev_win   # From previous script
)

# Output results
print("Expected Values if either team wins Game 3:")
print(f"EV if Team 1 wins Game 3: ${ev_if_team1_wins_game3:.2f}")
print(f"EV if Team 2 wins Game 3: ${ev_if_team2_wins_game3:.2f}")
print("\nGame 3 EV Delta:")
print(f"EV Delta: ${game3_ev_delta:.2f}")


Expected Values if either team wins Game 3:
EV if Team 1 wins Game 3: $22972.13
EV if Team 2 wins Game 3: $7747.85

Game 3 EV Delta:
EV Delta: $15224.29


In [6]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Automatically manage ChromeDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open a website
driver.get('https://www.espn.com/nfl/futures')

# Wait for page to load
time.sleep(5)  # Adjust based on your network speed


# Locate the odds table
try:
    # Update selector to match the table's structure
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

    chiefs_odds = None
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        team_name = cells[0].text.strip()
        odds = cells[1].text.strip()

        if team_name.lower() == "kansas city chiefs":
            chiefs_odds = odds
            break

    if chiefs_odds:
        print(f"The Kansas City Chiefs' Super Bowl odds are: {chiefs_odds}")
    else:
        print("Kansas City Chiefs' odds not found.")
finally:
    driver.quit()


IndexError: list index out of range

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Set up the WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

try:
    # Open the ESPN NFL Futures page
    driver.get('https://www.espn.com/nfl/futures')

    # Wait for the page to load (optional, adjust if needed)
    driver.implicitly_wait(10)

    # Locate the div containing the table
    futures_div = driver.find_element(By.CSS_SELECTOR, "div[data-testid='betFutures']")

    # Locate all rows in the table
    rows = futures_div.find_elements(By.CSS_SELECTOR, "table[data-testid='prism-Table'] tbody tr")

    chiefs_odds = None

    # Iterate through the rows to find the Kansas City Chiefs
    for row in rows:
        # Find the team name and odds
        team_name_element = row.find_element(By.CSS_SELECTOR, "a[data-testid='prism-linkbase'] span")
        odds_element = row.find_elements(By.CSS_SELECTOR, "a[data-testid='prism-linkbase']")[-1]  # Last link for odds
        
        team_name = team_name_element.text.strip()
        odds = odds_element.text.strip()

        if team_name.lower() == "kansas city chiefs":
            chiefs_odds = odds
            break

    if chiefs_odds:
        print(f"The Kansas City Chiefs' Super Bowl odds are: {chiefs_odds}")
    else:
        print("Kansas City Chiefs' odds not found.")

finally:
    driver.quit()


The Kansas City Chiefs' Super Bowl odds are: +375


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Set up the WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

try:
    # Open the ESPN NFL Futures page
    driver.get('https://www.espn.com/nfl/futures/_/group/conference')

    # Wait for the page to load
    driver.implicitly_wait(10)

    # Locate the section specifically for the "AFC Champion" odds
    afc_champion_section = driver.find_element(By.XPATH, "//div[contains(@class, 'ANPUN') and contains(@data-testid, 'betFutures')]")

    # Find the table within the AFC Champion section
    rows = afc_champion_section.find_elements(By.XPATH, ".//table[@data-testid='prism-Table']/tbody/tr")

    # Initialize variable to store Chiefs' odds
    chiefs_odds = None

    # Iterate through each row to find the Kansas City Chiefs and retrieve their odds
    for row in rows:
        # Extract team name and odds from the row
        team_name_element = row.find_element(By.XPATH, ".//a/span[contains(@class, 'xTell')]")
        odds_element = row.find_elements(By.XPATH, ".//a[@data-testid='prism-linkbase']")[-1]  # Last link for odds

        team_name = team_name_element.text.strip()
        odds = odds_element.text.strip()

        # Check if the team is the Kansas City Chiefs
        if team_name == "Kansas City Chiefs":
            chiefs_odds = odds
            break

    if chiefs_odds:
        print(f"The Kansas City Chiefs' Conference odds are: {chiefs_odds}")
    else:
        print("Kansas City Chiefs' odds not found in the AFC Champion section.")

finally:
    driver.quit()


The Kansas City Chiefs' Conference odds are: +160


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# Provide the path to ChromeDriver
service = Service('/Users/aceyvogelstein/Downloads/chromedriver')
driver = webdriver.Chrome(service=service)

# Open the website
driver.get('https://www.espn.com/nfl/futures')


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 128
Current browser version is 130.0.6723.117 with binary path /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
Stacktrace:
0   chromedriver                        0x0000000104cbd208 cxxbridge1$str$ptr + 1927396
1   chromedriver                        0x0000000104cb566c cxxbridge1$str$ptr + 1895752
2   chromedriver                        0x00000001048b0808 cxxbridge1$string$len + 89564
3   chromedriver                        0x00000001048e66f4 cxxbridge1$string$len + 310472
4   chromedriver                        0x00000001048e5940 cxxbridge1$string$len + 306964
5   chromedriver                        0x00000001048e1654 cxxbridge1$string$len + 289832
6   chromedriver                        0x00000001048ddae4 cxxbridge1$string$len + 274616
7   chromedriver                        0x000000010491e73c cxxbridge1$string$len + 539920
8   chromedriver                 

In [16]:
def calculate_ev(probability, payout):
    """Calculate expected value based on probability and payout."""
    return probability * payout

def calculate_conditional_ev(current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    """Calculate conditional EV if the team wins the current round."""
    conditional_conference_prob = current_conference_prob / current_round_prob
    conditional_champ_prob = current_champ_prob / current_round_prob
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)
    return conference_ev + champ_ev

# Inputs for the Tigers and Guardians
team1 = {  # Tigers
    "current_round_prob": 0.534,      
    "current_conference_prob": 0.18,      
    "current_champ_prob": 0.06,        
    "conference_payout": 0,             
    "champ_payout": 204452               
}

team2 = {  # Guardians
    "current_round_prob": 1 - team1["current_round_prob"],       
    "current_conference_prob": 0.15,      
    "current_champ_prob": 0.06,         
    "conference_payout": 0,     
    "champ_payout": 59160               
}

# Calculate conditional EVs if each team wins the current round
team1_ev_win = calculate_conditional_ev(
    team1["current_conference_prob"], team1["conference_payout"],
    team1["current_champ_prob"], team1["champ_payout"],
    team1["current_round_prob"]
)

team2_ev_win = calculate_conditional_ev(
    team2["current_conference_prob"], team2["conference_payout"],
    team2["current_champ_prob"], team2["champ_payout"],
    team2["current_round_prob"]
)

# Calculate the Round Delta as the difference between conditional EVs
series_delta = abs(team1_ev_win - team2_ev_win)

# Output results
print("Conditional Expected Values if Team Wins Current Round:")
print(f"Tigers EV if win round: ${team1_ev_win:.2f}")
print(f"Guardians EV if win round: ${team2_ev_win:.2f}")
print("\nSeries Delta:")
print(f"Series Delta: ${series_delta:.2f}")


Conditional Expected Values if Team Wins Current Round:
Tigers EV if win round: $22972.13
Guardians EV if win round: $7617.17

Series Delta:
Series Delta: $15354.97


In [ ]:
# loops thru each first-round NFL playoff matchup
# adds quarterfinals to the EV calculation



import mysql.connector
from collections import defaultdict

# Database connection setup
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get Potential Payouts for Conference, Championship, and Quarterfinals events
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.EventType IN ('Conference Winner', 'Championship', 'Quarterfinals')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Fetch and organize payouts in a dictionary
payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0, 'payout_quarterfinals': 0})

# Matchup structure with actual team names
matchups = {
    "AFC": [
        ("Kansas City Chiefs", "Miami Dolphins"),
        ("Cincinnati Bengals", "Pittsburgh Steelers"),
        ("Jacksonville Jaguars", "Cleveland Browns")
    ],
    "NFC": [
        ("San Francisco 49ers", "Minnesota Vikings"),
        ("Detroit Lions", "Atlanta Falcons"),
        ("Dallas Cowboys", "Seattle Seahawks")
    ]
}

# Define probabilities and payouts for each team
team_probabilities = {
    "Kansas City Chiefs": {'current_round_prob': 0.75, 'current_quarterfinals_prob': 0.3, 'current_conference_prob': 0.2, 'current_champ_prob': 0.1},
    "Miami Dolphins": {'current_round_prob': 0.25, 'current_quarterfinals_prob': 0.12, 'current_conference_prob': 0.1, 'current_champ_prob': 0.05},
    "Cincinnati Bengals": {'current_round_prob': 0.7, 'current_quarterfinals_prob': 0.3,'current_conference_prob': 0.18, 'current_champ_prob': 0.08},
    "Pittsburgh Steelers": {'current_round_prob': 0.3, 'current_quarterfinals_prob': 0.14, 'current_conference_prob': 0.08, 'current_champ_prob': 0.04},
    "Jacksonville Jaguars": {'current_round_prob': 0.6, 'current_quarterfinals_prob': 0.3,'current_conference_prob': 0.15, 'current_champ_prob': 0.07},
    "Cleveland Browns": {'current_round_prob': 0.4, 'current_quarterfinals_prob': 0.28, 'current_conference_prob': 0.09, 'current_champ_prob': 0.03},
    "San Francisco 49ers": {'current_round_prob': 0.75, 'current_quarterfinals_prob': 0.3,'current_conference_prob': 0.22, 'current_champ_prob': 0.12},
    "Minnesota Vikings": {'current_round_prob': 0.25, 'current_quarterfinals_prob': 0.12,'current_conference_prob': 0.11, 'current_champ_prob': 0.05},
    "Detroit Lions": {'current_round_prob': 0.65, 'current_quarterfinals_prob': 0.3,'current_conference_prob': 0.17, 'current_champ_prob': 0.07},
    "Atlanta Falcons": {'current_round_prob': 0.35, 'current_quarterfinals_prob': 0.16,'current_conference_prob': 0.08, 'current_champ_prob': 0.03},
    "Dallas Cowboys": {'current_round_prob': 0.6, 'current_quarterfinals_prob': 0.28,'current_conference_prob': 0.18, 'current_champ_prob': 0.08},
    "Seattle Seahawks": {'current_round_prob': 0.4, 'current_quarterfinals_prob': 0.18,'current_conference_prob': 0.09, 'current_champ_prob': 0.04}
}

# Populate payouts based on the fetched data
for participant_name, event_type, total_payout in cursor.fetchall():
    payouts[participant_name][f'payout_{event_type.lower().replace(" ", "_")}'] = float(total_payout)

cursor.close()
conn.close()

# Define EV calculation functions
def calculate_ev(probability, payout):
    return probability * payout

def calculate_conditional_ev(current_quarterfinals_prob, quarterfinals_payout, current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    conditional_quarterfinals_prob = current_quarterfinals_prob / current_round_prob
    conditional_conference_prob = current_conference_prob / current_round_prob
    conditional_champ_prob = current_champ_prob / current_round_prob
    
    quarterfinals_ev = calculate_ev(conditional_quarterfinals_prob, quarterfinals_payout)
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)
    
    return quarterfinals_ev + conference_ev + champ_ev

# Loop through each conference and matchup, calculating EVs
for conference, games in matchups.items():
    print(f"\n{conference} Conference First-Round Matchups:\n")
    for team1_name, team2_name in games:
        team1 = {
            "current_round_prob": team_probabilities[team1_name]["current_round_prob"],
            "current_quarterfinals_prob": team_probabilities[team1_name]["current_quarterfinals_prob"],
            "current_conference_prob": team_probabilities[team1_name]["current_conference_prob"],
            "current_champ_prob": team_probabilities[team1_name]["current_champ_prob"],
            "quarterfinals_payout": payouts[team1_name]["payout_quarterfinals"],
            "conference_payout": payouts[team1_name]["payout_conference"],
            "champ_payout": payouts[team1_name]["payout_championship"]
        }

        team2 = {
            "current_round_prob": team_probabilities[team2_name]["current_round_prob"],
            "current_quarterfinals_prob": team_probabilities[team2_name]["current_quarterfinals_prob"],
            "current_conference_prob": team_probabilities[team2_name]["current_conference_prob"],
            "current_champ_prob": team_probabilities[team2_name]["current_champ_prob"],
            "quarterfinals_payout": payouts[team2_name]["payout_quarterfinals"],
            "conference_payout": payouts[team2_name]["payout_conference"],
            "champ_payout": payouts[team2_name]["payout_championship"]
        }

        # Calculate conditional EVs
        team1_ev_win = calculate_conditional_ev(
            team1["current_quarterfinals_prob"], team1["quarterfinals_payout"],
            team1["current_conference_prob"], team1["conference_payout"],
            team1["current_champ_prob"], team1["champ_payout"],
            team1["current_round_prob"]
        )

        team2_ev_win = calculate_conditional_ev(
            team2["current_quarterfinals_prob"], team2["quarterfinals_payout"],
            team2["current_conference_prob"], team2["conference_payout"],
            team2["current_champ_prob"], team2["champ_payout"],
            team2["current_round_prob"]
        )

        # Calculate the Series Delta
        series_delta = abs(team1_ev_win - team2_ev_win)

        # Output results
        print(f"{team1_name} vs {team2_name}")
        print(f"{team1_name} EV if win round: ${team1_ev_win:.2f}")
        print(f"{team2_name} EV if win round: ${team2_ev_win:.2f}")
        print(f"Series Delta: ${series_delta:.2f}\n")


In [2]:
# Playoffs Round/Series EV Calculator (Isolated to the Matchup)


import mysql.connector
from collections import defaultdict

conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get Potential Payouts for Conf, Champ
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Fetch results and organize them in a dictionary
payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0})

# Define the relevant teams by full names
relevant_teams = [
    
    ## INSERT FULL TEAM NAMES HERE
    'Kansas City Chiefs', 'Baltimore Ravens'
]

# Fetch and organize the payouts by relevant teams
for participant_name, event_type, total_payout in cursor.fetchall():
    # Check if the participant name matches any relevant team
    if participant_name in relevant_teams:
        team_name = participant_name
        total_payout = float(total_payout)  # Convert to float for compatibility
        
        if event_type == 'Conference Winner':
            payouts[team_name]['payout_conference'] = total_payout
        elif event_type == 'Championship':
            payouts[team_name]['payout_championship'] = total_payout
        
        # Print out the potential payout for verification
        print(f"Team: {team_name}, Event Type: {event_type}, Total Payout: {total_payout}")

cursor.close()
conn.close()

# Define the EV calculation functions
def calculate_ev(probability, payout):
    """Calculate expected value based on probability and payout."""
    return probability * payout

def calculate_conditional_ev(current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    """Calculate conditional EV if the team wins the current round."""
    conditional_conference_prob = current_conference_prob / current_round_prob
    conditional_champ_prob = current_champ_prob / current_round_prob
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)
    return conference_ev + champ_ev

# Inputs for the two teams
team1 = { 
    
    ## INSERT PROBABILITIES BELOW
    
    "current_round_prob": 0.534,      
    "current_conference_prob": 0.18,      
    "current_champ_prob": 0.06,        
    "conference_payout": payouts['Detroit Tigers']['payout_conference'],             
    "champ_payout": payouts['Detroit Tigers']['payout_championship']               
}

team2 = { 
    "current_round_prob": 1 - team1["current_round_prob"],      
    
     ## INSERT PROBABILITIES BELOW
    
    "current_conference_prob": 0.15,    
    "current_champ_prob": 0.06,       
    "conference_payout": payouts['Cleveland Guardians']['payout_conference'],  
    "champ_payout": payouts['Cleveland Guardians']['payout_championship']   
}

# Calculate conditional EVs if each team wins the current round
team1_ev_win = calculate_conditional_ev(
    team1["current_conference_prob"], team1["conference_payout"],
    team1["current_champ_prob"], team1["champ_payout"],
    team1["current_round_prob"]
)

team2_ev_win = calculate_conditional_ev(
    team2["current_conference_prob"], team2["conference_payout"],
    team2["current_champ_prob"], team2["champ_payout"],
    team2["current_round_prob"]
)

# Calculate the Series Delta as the difference between conditional EVs
series_delta = abs(team1_ev_win - team2_ev_win)

# Output results
print("Conditional Expected Values if Team Wins Current Round:")
print(f"Baltimore Ravens EV if win round: ${team1_ev_win:.2f}")
print(f"Kansas City Chiefs EV if win round: ${team2_ev_win:.2f}")
print("\nSeries Delta:")
print(f"${series_delta:.2f}")


Conditional Expected Values if Team Wins Current Round:
Detroit Tigers EV if win round: $0.00
Cleveland Guardians EV if win round: $0.00

Series Delta:
$0.00
